In [1]:
# default_exp actorcritic.sac

In [2]:
#export
import torch.nn.utils as nn_utils
from fastai.torch_basics import *
from fastai.data.all import *
from fastai.basics import *
from dataclasses import field,asdict
from typing import List,Any,Dict,Callable
from collections import deque
import gym
import torch.multiprocessing as mp
from torch.optim import *

from fastrl.data import *
from fastrl.async_data import *
from fastrl.basic_agents import *
from fastrl.learner import *
from fastrl.metrics import *
from fastai.callback.progress import *

if IN_NOTEBOOK:
    from IPython import display
    import PIL.Image

In [3]:
# hide
from nbdev.showdoc import *
from nbdev.imports import *
from nbdev.export2html import *
if not os.environ.get("IN_TEST", None):
    assert IN_NOTEBOOK
    assert not IN_COLAB
    assert IN_IPYTHON

# SAC

> Soft Actor Critic

In [6]:
# export
class LinearA2C(Module):
    def __init__(self, input_shape, n_actions):
        super(LinearA2C, self).__init__()

        self.policy = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, n_actions)
        )

        self.value = nn.Sequential(
            nn.Linear(input_shape[0], 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def _get_conv_out(self, shape):
        o=self.conv(torch.zeros(1, *shape))
        return int(np.prod(o.size()))

    def forward(self,x):
        fx=x.float()
        return self.policy(fx),self.value(fx)

In [7]:
class SACModule(Module):pass

In [ ]:
# export
class SACTrainer(Callback):
    
    def after_backward(self):
        nn_utils.clip_grad_norm_(self.learn.model.parameters(),self.learn.clip_grad)

In [ ]:
# export
def loss_func(pred,yb,learn):

    

    return loss_v

class SACLearner(AgentLearner):
    def __init__(self,dls,**kwargs):
        self.create_m=True
        super().__init__(dls,loss_func=partial(loss_func,learn=self),**kwargs)
        
    def _split(self, b):
        if len(b)==1 and type(b[0])==tuple:b=b[0]
        super()._split(b)

In [ ]:
env='CartPole-v1'
model=LinearA2C((4,),2)


blk=IterableDataBlock(blocks=(FirstLastExperienceBlock(a=0,seed=0,n_steps=4,dls_kwargs={'bs':1,'num_workers':0,
                                                                             'verbose':False,'indexed':True,'shuffle_train':False,
                                                                             'batch_tfms':lambda x:(x['s'],x)})),
                      splitter=FuncSplitter(lambda x:False),
                      batch_tfms=lambda x:(x['s'],x),
                     )
dls=blk.dataloaders([env]*15,bs=128)

agent=ActorCriticAgent(model=model)
learner=SACLearner(dls,agent=agent,cbs=[A3CTrainer],metrics=[AvgEpisodeRewardMetric()])
learner.fit(400,lr=0.001,wd=0)

In [2]:
# hide
from nbdev.export import *
from nbdev.export2html import *
notebook2script()
notebook2html(n_workers=0)

Converted 00_core.ipynb.
Converted 01_wrappers.ipynb.
Converted 02_callbacks.ipynb.
Converted 03_basic_agents.ipynb.
Converted 04_learner.ipynb.
Converted 05a_data.ipynb.
Converted 05b_async_data.ipynb.
Converted 06_basic_train.ipynb.
Converted 12_a3c.a3c_data.ipynb.
Converted 13_metrics.ipynb.
Converted 14_actorcritic.sac.ipynb.
Converted index.ipynb.


converting: /opt/project/fastrl/nbs/14_actorcritic.sac.ipynb
